# NLP: Product Comment Sentiment Analysis

**Resources**
*   https://www.geeksforgeeks.org/what-is-sentiment-analysis/ (intro)
*   https://www.nltk.org/api/nltk.tokenize.html (tokenize - separate sentence into words)
*   https://youtu.be/9p1KYtYAus8 (vader lexicon tutorial - for sentiment analysis)
*   dataset (#56 from https://www.nltk.org/nltk_data/)
*   dataset downloaded from https://www.kaggle.com/datasets/mdwaquarazam/headphone-dataset-review-analysis

<br>


<br>
Documents (bg study, methods etc will do later)

> Step 1: Get data (csv file)

> Step 2: Pre-processing. Tokenize, Lemmatize, remove stopwords, punctuations etc, then do sentiment analysis

> Step 3: Count vectorizer (frequency each words appeared), then identify similarity (use cosine similarity equation, correlation etc.)

> Step 4: each of us will choose our preferred method (bayes, knn, k-means)

> Step 5: compare the results (from using different methods)







# Getting Data & Setup


In [18]:
%pip install nltk
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [1]:
# enter: pip install nltk  (when using python)

import nltk
import pandas as pd    # to allow us to read csv file
import random
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import subprocess
import re

In [2]:
# install nltk (terminal line commands)
%pip install nltk
%pip install pandas

import nltk
import site
import pandas as pd    # to allow us to read csv file
import random
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize # for tokenization of words
from nltk.stem import WordNetLemmatizer # for lemmatization
import re # python's 're' module to create empty strings with pattern that matches pucnctuation marks
import string

nltk.download('all')
site.getsitepackages() # to ourpur current path, we imported site, and getsitepackages()

You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\Users\User\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\U

KeyboardInterrupt: 

# Read data

In [25]:
data = pd.read_csv('headphone_datn.csv')

In [14]:
# data.head()
total = data.count()
print(total)

Customer_Name    1604
REVIEW_TITLE     1594
Color            1604
REVIEW_DATE      1604
COMMENTS         1546
RATINGS          1604
dtype: int64


# Pre-Processing

In [26]:
# create pre processing function

def preprocess_text(text):
  # remove punctuations
  data['COMMENTS'].dropna(inplace=True)
  data['COMMENTS'] = data['COMMENTS'].astype(str)

  tokens = nltk.word_tokenize(text)
  # return [w for w in tokens if w.isalpha()]   
  # tokens = nltk.word_tokenize(df['COMMENTS'])

  # tokenize the text 
  tokens = word_tokenize(cleaned_text)
  return tokens
  # # remove stop words
  # filtered_tokens = [token for token in tokens if token not in stopwords.words('english')]

  # # lemmatize the tokens
  # lemmatizer = WordNetLemmatizer()
  # lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

  # #Join the tokens back into a string
  # processed_text = ' '.join(lemmatized_tokens)
  # return processed_text


data['tokenized'] = data.apply(lambda x: preprocess_text(x['COMMENTS']), axis=1) 
print(data['tokenized'])


0       Okay .. I skeptical first buy tight budget , h...
1       The earphone worth pay . The design earbud may...
2       Particularly people sensitive ear , get ear pa...
3       Built Quality 6.5 10The lower wire durable nyl...
4       Do n't go 4 start rating . This boat brand end...
                              ...                        
1599    Quite good sound qualityAnd impressively good ...
1600                                                  Osm
1601    Earphones fit well onto ear , n't block ear ou...
1602    Sound quality bad ... Over bad product ... Not...
1603    This call ... Mic good ... But Comfort bad mus...
Name: tokenized, Length: 1604, dtype: object


In [10]:
analyzer = SentimentIntensityAnalyzer()

# create get_sentiment function

def get_sentiment(text):

    scores = analyzer.polarity_scores(text)
    sentiment = 1 if scores['pos'] > 0 else 0

    return sentiment

# apply get_sentiment function

data['sentiment'] = data['COMMENTS'].apply(get_sentiment)



NameError: name 'data' is not defined